In [1]:
import pandas as pd
import time

In [2]:
BASE_DIR = 'G:/Program Files/PycharmProjects/job_analytics/jobtitle_graph/project_SkillSense/app_SkillSense/'
DATA_DIR = BASE_DIR + 'data/'

chord_nodes_df = pd.read_csv(DATA_DIR + '/node_TopicSkill.csv', keep_default_na=False)
ch_label_dict = dict(zip(chord_nodes_df.id, chord_nodes_df.label))

query_TopicOnly = 'nodeType == "{}"'.format('Topic')
query_SkillOnly = 'nodeType == "{}"'.format('Skill')

skillgroup_df = chord_nodes_df.query(query_TopicOnly)
skills_df = chord_nodes_df.query(query_SkillOnly)

skillgroup_id_list = sorted(skillgroup_df.id.tolist())
skills_id_list = sorted(skills_df.id.tolist())

ch_topicskills_linksIDF_df = pd.read_csv(DATA_DIR + '/edge_TopicSkillIDF.csv', keep_default_na=False)

In [13]:
# to rename Skills Group according to new representative skills
def prepNewTopicName():
    sg_s_optm_df = pd.read_csv(DATA_DIR + '/edge_TopicSkillOneRepSkill_46_20180104-1630.csv', keep_default_na=False)
    sg_s_optm_df.sort_values(['source', 'weight'], ascending=[True,False], inplace=True)
    master_list = []
    for skill_group_id in skillgroup_id_list:
        cores_skill_group_filter = sg_s_optm_df['source'] == skill_group_id
        selected_sg_df = sg_s_optm_df[cores_skill_group_filter].head(5).copy()
        if selected_sg_df.shape[0] == 0:
            continue
#         print skill_group_id
        skill_id_list = selected_sg_df.target.tolist()
        skill_list = [ch_label_dict[int(skill_id)] for skill_id in skill_id_list]
        s = ", ";
        newSGName = s.join(skill_list)
        master_list.append([newSGName])

    onerep_sg_s_df = pd.DataFrame(master_list, columns=['skill_group_name_NEW'])
    timestr = time.strftime("%Y%m%d-%H%M")
    onerep_sg_s_df.to_csv(DATA_DIR + '/node_TopicSkill(SGNamesNEW)'+timestr+'.csv', index=False)

    return onerep_sg_s_df

In [7]:
# to assign one skill to only one skill group, from each mapping to SG, take highest weight:
def prepNewRepSkills():
    master_list = []
    skills_wo_group = []
    ch_topicskills_linksIDF_df.sort_values(['target', 'weight'], ascending=[True,False], inplace=True)
    for selector_skillid in skills_id_list:
        filterbyskill = ch_topicskills_linksIDF_df['target'] == selector_skillid
        skillrows = ch_topicskills_linksIDF_df[filterbyskill].copy()
        if skillrows.shape[0] == 0:
            skills_wo_group.append([selector_skillid, ch_label_dict[selector_skillid]])
            continue
        master_list.append([skillrows.iloc[0].source, skillrows.iloc[0].target, skillrows.iloc[0].weight])
#         print ch_label_dict[selector_skillid]

    skills_wo_grp_df = pd.DataFrame(skills_wo_group, columns=['id', 'skill_name'])
    
    timestr = time.strftime("%Y%m%d-%H%M")
    fname = 'dict_Skills_wo_group'+timestr+'.csv'
    # skills_wo_grp_df.to_csv(DATA_DIR + '/' + fname, index=False)

    onerep_sg_s_df = pd.DataFrame(master_list, columns=['source', 'target', 'weight'])
    onerep_sg_s_df.sort_values(['target', 'weight'], ascending=[True,False], inplace=True)
    fname2 = 'edge_TopicSkillOneRepSkill_'+ str(len(skillgroup_id_list)) +'_'+timestr+'.csv'
    onerep_sg_s_df.to_csv(DATA_DIR + '/'+ fname2, index=False)

    return onerep_sg_s_df

In [8]:
prepNewRepSkills()

,source,target,weight
0,33.0,67.0,0.072134
1,19.0,74.0,0.132696
2,16.0,76.0,0.405073
3,16.0,77.0,0.082209
4,22.0,79.0,0.062651
5,27.0,80.0,0.065412
6,16.0,84.0,0.229011
7,19.0,86.0,0.079894
8,9.0,89.0,0.351749
9,51.0,98.0,1.344834


In [14]:
prepNewTopicName()

,skill_group_name_NEW
0,"Java, Javascript, Sql, Web, J2Ee"
1,"Electrical, Mechanical, Electrical Engineering..."
2,"Accounting, Acca, Accountancy, Lcci, Accountin..."
3,"Advertising, Media, Social Media, Digital Mark..."
4,"Presentation, Business Development, Presentati..."
5,"Customer Service, Etiquette, Service Excellenc..."
6,"Driving, Motorcycle, Sat, Route, Road"
7,"Logistics, Freight, Supply Chain, Supply, Forw..."
8,"Clinical, Medical, Medicine, Physicians, Accre..."
9,"Restaurant, Food, Food And Beverage, Wine, Res..."
